In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_excel('E-743 Amount Distribution.xlsx', usecols = 'A:C', skiprows = 1)
df.head()

,Name,Months,Amount
0,Carolyn,"1, 3, 10",90
1,Aaron,7,12
2,Catherine,"5, 6, 7, 8",100
3,Nathan,"2, 9, 12",24
4,Donna,"11, 12",78


In [3]:
df = (
    df
    .assign(Months = df['Months'].str.split(', ')).explode('Months')
    .pipe(
        lambda row: row.assign(
            Amount = row['Amount'] / 
                row.groupby('Name')['Name'].transform('count')
        )
    )
    .astype({'Months': int})
)
df.head()

,Name,Months,Amount
0,Carolyn,1,30.0
0,Carolyn,3,30.0
0,Carolyn,10,30.0
1,Aaron,7,12.0
2,Catherine,5,25.0


In [4]:
names = df['Name'].unique()
months = list(range(1, 13))

crossjoin_merge = (
    pd.MultiIndex.from_product(
        [names, months], 
        names = ['Name', 'Months']
    ).to_frame(index = False)
    .merge(df, on = ['Name', 'Months'], how = 'left')
    .fillna(0)
)
crossjoin_merge.head(15)

,Name,Months,Amount
0,Carolyn,1,30.0
1,Carolyn,2,0.0
2,Carolyn,3,30.0
3,Carolyn,4,0.0
4,Carolyn,5,0.0
5,Carolyn,6,0.0
6,Carolyn,7,0.0
7,Carolyn,8,0.0
8,Carolyn,9,0.0
9,Carolyn,10,30.0


In [5]:
pivot = (
    crossjoin_merge
    .pivot_table(
        index = 'Name', columns = 'Months', 
        values = 'Amount', aggfunc = 'sum'
    )
    .reset_index()
)
pivot.columns.name = None
pivot

,Name,1,2,3,4,5,6,7,8,9,10,11,12
0,Aaron,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
1,Carolyn,30.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0
2,Catherine,0.0,0.0,0.0,0.0,25.0,25.0,25.0,25.0,0.0,0.0,0.0,0.0
3,Donna,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,39.0
4,Nathan,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,8.0


In [6]:
col_names = ['Name'] + list(
    pd.to_datetime(
        pivot.columns.tolist()[1:], 
        format = '%m'
    ).strftime('%b')
)

pivot.columns = col_names
pivot

,Name,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,Aaron,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
1,Carolyn,30.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0
2,Catherine,0.0,0.0,0.0,0.0,25.0,25.0,25.0,25.0,0.0,0.0,0.0,0.0
3,Donna,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,39.0
4,Nathan,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,8.0


In [7]:
name_order = df['Name'].drop_duplicates()

pivot['Name'] = pd.Categorical(
    pivot['Name'], 
    categories = name_order, 
    ordered = True
)

pivot.sort_values('Name').reset_index(drop = True)

,Name,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,Carolyn,30.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0
1,Aaron,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
2,Catherine,0.0,0.0,0.0,0.0,25.0,25.0,25.0,25.0,0.0,0.0,0.0,0.0
3,Nathan,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,8.0
4,Donna,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,39.0
